In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


First run plain question through LLM.
 - Some movie plots may not be available or well represented in the general model.

In [2]:
import os
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import GPT4All

# load local model
llm = GPT4All(model='nous-hermes-llama2-13b.Q4_0.gguf')
template = """Question: {question}

Answer including relevant infos for movie, e.g. genre, director, etc.:"""

prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [3]:
question = """Tell me about the movie \"A Touch of Zen\"."""
llm_chain.invoke(question)

{'question': 'Tell me about the movie "A Touch of Zen".',
 'text': ' A Touch of Zen is a 1971 Taiwanese wuxia film directed by King Hu and starring Feng Hsu, Tien Ching, and Yu-Hua Chang. The film follows the story of Yang, an artist who becomes entwined in political intrigue when he meets Wu Kong, a mysterious swordsman with ties to the martial arts world. A Touch of Zen is widely regarded as one of the greatest films ever made and has had a significant influence on the wuxia genre.'}

LLM response might contain "hallucinations". 
E.g. the movie was released in 1971 and not 2001.

With Retrieval Augmented Generation the LLM answer can be improved.

<img src="images/RAG.png" alt="Retrieval Augmented Generation" width="30%" height="auto" class="blog-image">

Add domain specific context from [data](https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots/) containing ca. 35k movie plots

<img src="images/data-card.png" alt="Retrieval Augmented Generation" width="40%" height="auto" class="blog-image">

- save each movie plot as a separate document

In [4]:
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import DirectoryLoader

def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

Create Vectorstore once
 - movie data is stored as embedding vectors for **semantic** search
 - Vectorstore index is persisted in folder faiss_index

In [5]:
plots = pd.read_csv('wiki_movie_plots_deduped.csv')
for p in plots['Plot'][:3]:
    print(p)
    print()

A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]

The moon, painted with a smiling face hangs over a park at night. A young couple walking past a fence learn on a railing and look up. The moon smiles. They embrace, and the moon's smile gets bigger. They then sit down on a bench by a tree. The moon's view is blocked, causing him to frown. In the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything better.

The film, just over a minute long, is composed of two shots

In [6]:
# Store each movie plot as separate text file into folder
os.makedirs('documents', exist_ok=True)

for i, row in plots.iterrows():
    with open(f'documents/plot_{i}.txt', 'w') as f:
        txt = f"""
Title: {str(row['Title'])}
Release Year: {str(row['Release Year'])}
Genre: {str(row['Genre'])}
Ethnicity: {str(row['Origin/Ethnicity'])}
Director: {str(row['Director'])}
Cast: {str(row['Cast'])}
Plot: {str(row['Plot'])}
        """
        f.write(txt)

In [7]:
def create_index(directory):
    print("Loader...")
    loader = DirectoryLoader(directory)
    docs = loader.load()

    print("creating index")

    db = FAISS.from_documents(docs, embeddings)
    print("Saving local index...")
    db.save_local("faiss_index")

In [8]:
embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")  # downloads to ~/.cache 
# Following line needs about 1h to create; and 3 to load.

#Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.


#create_index('documents')  # initial vectore store creation. Comment out this line after index is created.
db = FAISS.load_local("./faiss_index", embeddings, allow_dangerous_deserialization=True)

Extract movie infos
 - that match the question
 - use as context info (automated Prompt Engineering)

In [9]:
# compress if model has small token window size (2k)
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.vectorstores import Chroma

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=db.as_retriever())
# A touch of Zen = plot_22997.txt
compressed_docs = compression_retriever.get_relevant_documents(question + " Summarize the plot. In addition, state Title, Release Year, and Director.")
pretty_print_docs(compressed_docs)
compressed_context = Chroma.from_documents(compressed_docs, embeddings)

/Users/germaingirndt/installs/anaconda/anaconda3/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/germaingirndt/installs/anaconda/anaconda3/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/germaingirndt/installs/anaconda/anaconda3/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
LLaMA ERROR: The prompt is 2268 tokens and the context window is 2048!
/Users/germaingirndt/installs/anaconda/anaconda3/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.war

Document 1:

Title: A Touch of Zen Release Year: 1971 Genre: unknown Ethnicity: Hong Kong Director: King Hu Cast: nan Plot: A remote mountain village in Ming China, the 14th century AD. [a] The story is largely seen through the eyes of Gu, who is a well-meaning but unambitious scholar and painter, with a tendency towards being clumsy and ineffectual.
----------------------------------------------------------------------------------------------------
Document 2:

Title - Zenobia Release Year - 1939 Genre - comedy Ethnicity - American Director - Gordon Douglas Cast - Oliver Hardy, Harry Langdon, Billie Burke Plot - Dr. Henry Tibbett is a country doctor who is called on by a travelling circus trainer to cure his sick elephant. After the doctor heals the grateful beast, the elephant becomes so attached to him that it starts to follow him everywhere. This leads to the trainer suing Dr. Tibbett for alienation of affection.
---------------------------------------------------------------------

In [10]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=compressed_context.as_retriever())
qa_chain.run({"query": question})

/Users/germaingirndt/installs/anaconda/anaconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' A Touch of Zen is a 1971 Hong Kong film directed by King Hu. It is considered one of the greatest films in Chinese cinema history, known for its innovative use of wuxia elements and its influence on later martial arts films. The story takes place in Ming China during the 14th century AD and follows a well-meaning but unambitious scholar named Gu who becomes entangled in a web of intrigue involving a mysterious woman, an evil monk, and a group of bandits. The film is known for its stunning cinematography, complex plot, and epic battle scenes.'

<!-- import gpt4all
model = gpt4all.GPT4All(model_name="ggml-mpt-7b-chat.bin") -->